In [2]:
#from /Users/hph/Dropbox/astrophys/juno/CMZoom_scripts import CMZoom_utility as cmz
from astropy.io import fits
import numpy as np
import matplotlib.pyplot as plt
from astropy import units as u
#import astrodendro
import os
from astropy import wcs
#from astrodendro import Dendrogram, pp_cataloga
#from astrodendro.analysis import PPStatistic
from astropy.table import Table, hstack, Column
from astropy.utils.console import ProgressBar
import reproject
print(np.__version__)
#import aplpy
import csv
import pandas as pd
import CMZoom_utility as cmz
%config Completer.use_jedi = False
# from IPython.core.display import display, HTML
# display(HTML("<style>.container {width:95% !important}</style>"))

%matplotlib inline

1.25.2


In [3]:
##################################################################################################
### setup 
##################################################################################################
### picks for the accurate catalog
rms = 3.0e6
dend_min_v = 3
dend_min_d = 1
dend_min_pix = 17
prune_min_peak = 6
prune_min_mean = 2
path = '/Users/hph/Dropbox/astrophys/CMZoom/cmzoom_catalog/catalog_acc/'
savepath = '/Users/hph/Dropbox/astrophys/CMZoom/SF_tracer_paper/'

### Here's the old catalog WITH SGRC ERROR!!!
#catalog = Table.read(path+'catalog_acc.fits')
catalog = Table.read(path+'teamcat.fits')

#######################################
### picks for the complete catalog
# rms = 3.0e6
# dend_min_v = 3
# dend_min_d = 1
# dend_min_pix = 17
# prune_min_peak = 4
# prune_min_mean = 2
# path = '/Users/hph/cmzoom_catalog/catalog_complete/'
# catalog = Table.read(path+'catalog_complete.fits')
# writefile = '/Users/hph/cmzoom_catalog/megacatalog_complete/catalog_complete.fits'
#######################################

date = '_02-02-20'
targetname = '_rms3e6_k14_dv'+str(dend_min_v)+'_dd'+str(dend_min_d)+'_dp'+str(dend_min_pix)+'_pp'+str(prune_min_peak)+'_pm'+str(prune_min_mean)+'_gal'+date

sma_path = path
herschel_path = path
catalog_path = path
figure_path = savepath

region_names = np.array(['G1.683-0.089', 
                      'G1.670-0.130', 
                      'G1.651-0.050', 
                      'G1.602+0.018',
                      'G1.085-0.027', 
                      'G1.038-0.074', 
                      'G0.891-0.048',
                      'G0.714-0.100',
                      'G0.699-0.028',
                      'G0.619+0.012',
                      'G0.489+0.010',
                      'G0.412+0.052',
                      'G0.393-0.034',
                      'G0.380+0.050',
                      'G0.340+0.055',
                      'G0.326-0.085',
                      'G0.316-0.201',
                      'G0.253+0.016',
                      'G0.212-0.001',
                      'G0.145-0.086',
                      'G0.106-0.082',
                      'G0.070-0.035',
                      'G0.068-0.075',
                      'G0.054+0.027',
                      'G0.014+0.021',
                      'G0.001-0.058',
                      'G359.948-0.052',
                      'G359.889-0.093',
                      'G359.865+0.022',
                      'G359.734+0.002',
                      'G359.648-0.133',
                      'G359.611+0.018',
                      'G359.615-0.243',
                      'G359.484-0.132',
                      'G359.137+0.031']) #'G359.863-0.069', cut out
cnames = np.array(['',
                 '',
                 '',
                 '',
                 '',
                 '',
                 '',
                 '',
                 'Sgr B2',
                 'Sgr B2 SE',
                 'Dust Ridge Clouds E\\&F',
                 'Dust Ridge Cloud D',
                 '',
                 'Dust Ridge Cloud C',
                 'Dust Ridge Cloud B',
                 'The Sailfish',
                 '',
                 'The Brick / The Lima Bean Cloud',
                 '',
                 'The Straw Cloud',
                 'The Sticks Cloud',
                 '',
                 'The Stone Cloud',
                 '',
                 '',
                 'The 20 km/s cloud',
                 'Sgr A*, circumnuclear disk',
                 'The 50 km/s cloud',
                 '',
                 '',
                 '',
                 '',
                 '',
                 'Sgr C',
                 '']) # '', cut out


path = '/Users/hph/Dropbox/astrophys/CMZoom/cmzoom_catalog/catalog_acc/'

#catalog = Table.read(path+'megacatalog_acc.fits')
#teamcat = Table.read(path+'megacatalog_team_acc_final_say.fits')
#### those catalogs have errors with sgrc ,use this instead
teamcat = Table.read(path+'teamcat.fits')


print(teamcat.keys())

['index', 'area_ellipse', 'area_exact', 'flux_integrated', 'major_sigma', 'minor_sigma', 'position_angle', 'radius', 'glon', 'glat', 'r_eff', 'flux_integrated_bgsub', 'min_flux_integrated', 'noise', 'peak_cont_flux', 'min_cont_flux', 'mean_cont_flux', 'npix', 'DustTemperature', 'herschel_column', 'mass', 'sma_column', 'r_eff_pc', 'Sigma', 'n', 'rho', 'tff', 'glon_0', 'mass_bgsub', 'suspicious', 'leaf_ID', 'mask_num', 'mass_unc', 'N_unc', 'n_unc', 'rho_unc', 'tff_unc', 'mass_bgsub_unc', 'atoll_final_say', 'SF_final_say', '8um_point_hph', '20-24um_point_hph', '70um_point_hph', 'YSO_hph', 'Walsh+2014_h2o_maser_hph', 'Caswell+2012_methanol_maser_hph', 'Gutermuth+2015_24um_point_hph', 'Molinari+2016_70um_point_hph', 'SF_any_all_hph', 'SF_any_certain_hph', '8um_point_brian', '20-24um_point_brian', '70um_point_brian', 'YSO_brian', 'Walsh+2014_h2o_maser_brian', 'Caswell+2012_methanol_maser_brian', 'Gutermuth+2015_24um_point_brian', 'Molinari+2016_70um_point_brian', 'SF_any_all_brian', 'SF_any_

In [4]:
###################################################
### cloud by cloud sf properties
###################################################
sfr_robust_bycloud = np.zeros(36)
sfr_any_bycloud = np.zeros(36)

cloud_mean_tff_robust = np.zeros(36)
cloud_mean_tff_amb = np.zeros(36)

cloud_Nsf_robust = np.zeros(36)
cloud_Fracsf_robust = np.zeros(36)

cloud_Nsf_amb = np.zeros(36)
cloud_Fracsf_amb = np.zeros(36)

### calculate IMF extrapolation 
import imf
kroupa = imf.Kroupa()
### how big are we allowing our biggest star to be?
mmax = 200
### what mass do consider "high mass" (pls no arguing)
cutoff1 = 8
over8fraction = (kroupa.m_integrate(cutoff1, mmax)[0] /
                 kroupa.m_integrate(kroupa.mmin, mmax)[0])

sfr_robust_Td_cat = np.zeros(36)

sfr_amb_Td_cat = np.zeros(36)


dsfr_rand_robust_Td_cat = np.zeros(36)
dsfrm_dust_robust_Td_cat = np.zeros(36)
dsfrp_dust_robust_Td_cat = np.zeros(36)
dsfr_sfe_robust_Td_cat = np.zeros(36)

dsfr_rand_amb_Td_50 = np.zeros(36)
dsfrm_dust_amb_Td_50 = np.zeros(36)
dsfrp_dust_amb_Td_50 = np.zeros(36)
dsfr_sfe_amb_Td_50 = np.zeros(36)

dsfr_rand_amb_Td_cat = np.zeros(36)
dsfrm_dust_amb_Td_cat = np.zeros(36)
dsfrp_dust_amb_Td_cat = np.zeros(36)
dsfr_sfe_amb_Td_cat = np.zeros(36)

### custom SFRs for Sgr B2 from Ginsburg+2018
sgrb2_sfr = 0.035
sgrb2_sfr_max = 0.062

sfe = 0.25
dsfe= 0.15
Td_l = 20
Td_m = 50
Td_h = 100

### calculate upper masses
mass_Td_cat = teamcat['mass']

for i in range(36):
    ccond = np.where(((teamcat['SF_final_say']==1)|(teamcat['SF_final_say']==1))&(teamcat['mask_num']!=27) & 
                     (catalog['mask_num']==i+1) & (teamcat['mask_num']!=9) & 
                     (teamcat['mask_num']!=13) & (teamcat['mask_num']!=17) &
                     (teamcat['mask_num']!=33))[0]
    ambcond = np.where(((teamcat['SF_final_say']==1)|(teamcat['SF_final_say']==2))&(teamcat['mask_num']!=27) & 
                     (catalog['mask_num']==i+1) & (teamcat['mask_num']!=9) & 
                     (teamcat['mask_num']!=13) & (teamcat['mask_num']!=17) &
                     (teamcat['mask_num']!=33))[0]
     
    cloud_Nsf_robust[i]=len(ccond)
    cloud_Nsf_amb[i]=len(ambcond)
    
    if len(np.where(catalog['mask_num']==i+1)[0])!=0:
        cloud_Fracsf_robust[i]=float(len(ccond))/float(len(np.where(catalog['mask_num']==i+1)[0]))
        cloud_Fracsf_amb[i]=float(len(ambcond))/float(len(np.where(catalog['mask_num']==i+1)[0]))

    sfr_robust_Td_cat[i] = (cmz.sin1yr/2.0e30)*np.sqrt(8.*cmz.G/(np.pi**2))*(sfe/over8fraction)*np.sum(np.sqrt((mass_Td_cat[ccond]*2.0e30)**3/(teamcat['r_eff_pc'][ccond]*3.1e16)**3))

    sfr_amb_Td_cat[i] = (cmz.sin1yr/2.0e30)*np.sqrt(8.*cmz.G/(np.pi**2))*(sfe/over8fraction)*np.sum(np.sqrt((mass_Td_cat[ambcond]*2.0e30)**3/(teamcat['r_eff_pc'][ambcond]*3.1e16)**3))
 
    dsfr_rand_robust_Td_cat[i] = (cmz.sin1yr/2.0e30)*np.sqrt(8.*cmz.G/(np.pi**2))*(sfe/over8fraction)*(3./2.)*np.sqrt(np.sum(((teamcat['mass_unc'][ccond]*2.0e30)*(mass_Td_cat[ccond]*2.0e30)**(1./2.)/(teamcat['r_eff_pc'][ccond]*3.1e16)**(3./2.))**2))
    dsfrm_dust_robust_Td_cat[i] = (cmz.sin1yr/2.0e30)*np.sqrt(8.*cmz.G/(np.pi**2))*(sfe/over8fraction)*(3./2.)*np.sqrt(np.sum(((0.5*mass_Td_cat[ccond]*2.0e30)*(mass_Td_cat[ccond]*2.0e30)**(1./2.)/(teamcat['r_eff_pc'][ccond]*3.1e16)**(3./2.))**2))
    dsfrp_dust_robust_Td_cat[i] = (cmz.sin1yr/2.0e30)*np.sqrt(8.*cmz.G/(np.pi**2))*(sfe/over8fraction)*(3./2.)*np.sqrt(np.sum(((1.0*mass_Td_cat[ccond]*2.0e30)*(mass_Td_cat[ccond]*2.0e30)**(1./2.)/(teamcat['r_eff_pc'][ccond]*3.1e16)**(3./2.))**2))                            
    dsfr_sfe_robust_Td_cat[i] = (cmz.sin1yr/2.0e30)*np.sqrt(8.*cmz.G/(np.pi**2))*(dsfe/over8fraction)*np.sum(np.sqrt((mass_Td_cat[ccond]*2.0e30)**3./(teamcat['r_eff_pc'][ccond]*3.1e16)**3.))
    
    dsfr_rand_amb_Td_cat[i] = (cmz.sin1yr/2.0e30)*np.sqrt(8.*cmz.G/(np.pi**2))*(sfe/over8fraction)*(3./2.)*np.sqrt(np.sum(((teamcat['mass_unc'][ambcond]*2.0e30)*(mass_Td_cat[ambcond]*2.0e30)**(1./2.)/(teamcat['r_eff_pc'][ambcond]*3.1e16)**(3./2.))**2))
    dsfrm_dust_amb_Td_cat[i] = (cmz.sin1yr/2.0e30)*np.sqrt(8.*cmz.G/(np.pi**2))*(sfe/over8fraction)*(3./2.)*np.sqrt(np.sum(((0.5*mass_Td_cat[ambcond]*2.0e30)*(mass_Td_cat[ambcond]*2.0e30)**(1./2.)/(teamcat['r_eff_pc'][ambcond]*3.1e16)**(3./2.))**2))
    dsfrp_dust_amb_Td_cat[i] = (cmz.sin1yr/2.0e30)*np.sqrt(8.*cmz.G/(np.pi**2))*(sfe/over8fraction)*(3./2.)*np.sqrt(np.sum(((1.0*mass_Td_cat[ambcond]*2.0e30)*(mass_Td_cat[ambcond]*2.0e30)**(1./2.)/(teamcat['r_eff_pc'][ambcond]*3.1e16)**(3./2.))**2))                            
    dsfr_sfe_amb_Td_cat[i] = (cmz.sin1yr/2.0e30)*np.sqrt(8.*cmz.G/(np.pi**2))*(dsfe/over8fraction)*np.sum(np.sqrt((mass_Td_cat[ambcond]*2.0e30)**3./(teamcat['r_eff_pc'][ambcond]*3.1e16)**3.))

    
cmz_sfr_robust_Td_cat = 0.
cmz_sfr_amb_Td_cat = 0.

### loop through CMZ clouds and sum up
for i in range(36):
    if (i+1!=9):
        cmz_sfr_robust_Td_cat+=sfr_robust_Td_cat[i]
        cmz_sfr_amb_Td_cat+=sfr_amb_Td_cat[i]

cmz_sfr_robust_Td_cat = cmz_sfr_robust_Td_cat + sgrb2_sfr
cmz_sfr_amb_Td_cat = cmz_sfr_amb_Td_cat + sgrb2_sfr

print(f"Robust: {cmz_sfr_robust_Td_cat}")
print(f"Ambiguous: {cmz_sfr_amb_Td_cat}")



Robust: 0.2429040337314883
Ambiguous: 0.29535946104189137
